In [1]:
import webbrowser
import pandas as pd
import gspread
import gspread_dataframe as gd
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials # Đọc dữ liệu từ googlesheets
from df2gspread import df2gspread as d2g # Ghi dữ liệu lên googlesheets
from pprint import pprint
from googleapiclient import discovery
import pypyodbc #connect với sql server
import numpy as np #Tính toán (=,-,*,/)
import sqlalchemy #query trực tiếp
import urllib #connect url
import time #
import datetime as dt
import pymssql
import random
import math
import calendar
import warnings
import itertools
warnings.filterwarnings("ignore")
import pygsheets

In [2]:
#connect server
db = pypyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=103.69.193.246;"
                        "Database=dwh;"
                        "uid=dwh;pwd=F88!23456789")
print('Connected to SQL server: DWH')

Connected to SQL server: DWH


In [3]:
scope = ["https://spreadsheets.google.com/feeds",'https://www.googleapis.com/auth/spreadsheets',"https://www.googleapis.com/auth/drive.file","https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name(r'D:\F88\Python code\Immediate_warning\service_account.json', scope)
gc = gspread.authorize(credentials)

In [4]:
client = pygsheets.authorize(service_account_file=r'D:\F88\Python code\Immediate_warning\service_account.json')

In [387]:
#Lấy ID và truy cập wb Backup
backup_ss_id='1iYODUg3bS0JlRzkuq-cnrVZYXJgcNGAa2Wj7i2McHIc'
backup_wb=gc.open_by_key(backup_ss_id)

#Lấy ID và truy cập wb Warning.Control
control_ssheet_id= '1101WzQvQ897hq4xyjvKHesT4ycx8to7bD9EvLGkcPSQ' 
control_wb=gc.open_by_key(control_ssheet_id)

#Function chuyển sheet thành Data Frame
def to_dataframe(wb_name,sheet_name,data_row,col):
    a=wb_name.worksheet(sheet_name).get_all_values()
    return pd.DataFrame(data=a[data_row:],columns=a[col])

#Funtion update data vào sheet wb_name:tên wb,target_sheet: tên sheet cần update; data:dữ liệu muốn update
def update_data(wb_name,target_sheet,data):
    existing =to_dataframe(wb_name,target_sheet,1,0)
    updated = existing.append(data,sort=False)
    gd.set_with_dataframe(wb_name.worksheet(target_sheet),updated)

# Function ghi đè dữ liệu lên gg sheet
def write_data(wb_name,ws_name,ws_range,data):
    wb_name.values_clear(ws_range)
    sheet=wb_name.worksheet(ws_name)
    set_with_dataframe(sheet,data)
    print('Đã ghi dữ liệu lên sheet '+ ws_name)

#Function backup data các HĐ đã check
def back_up_data(data):
    update_data(backup_wb,'backup',data[['codeno','reason_check','shopcode','year_month']])
    short_list=to_dataframe(control_wb,'short_list',1,0)

In [6]:
# collecting shop information from business department
shop_id='1OCzHxTE7Er8_W1-IT92rTOCISAlufFOjcwtw2eIX0tU'
shop_wb=client.open_by_key(shop_id)
shop_data=shop_wb.worksheet_by_title('Everything').get_as_df()

In [110]:
shop_data = shop_data[['Mã PGD', 'Tỉnh/TP']]

In [105]:
# collecting shop information from MKT department
shop_id_mkt='1ZTQE_pfBCAUr-0GSMPJBqtCyZVxX4hdiPuT_RDrdjeQ'
shop_wb_mkt=client.open_by_key(shop_id_mkt)
shop_data_mkt=shop_wb_mkt.worksheet_by_title('shop_detail').get_as_df()

In [108]:
shop_data_mkt = shop_data_mkt[['PGD', 'Mã PGD', 'GĐM', 'TPK', 'QLKV']]

In [98]:
open_close_before_after_cash_count.head(5)

,shop_id,shop_name,cash_count_date,min_cash_count,max_cash_count,shopcode,codeno,customer,category_name,openhour,closehour,pawn_fraud_group
0,22,F88 580 Trường Chinh,2020-11-08,2020-11-08 08:11:56.453,2020-11-08 21:00:41.930,HNTC,HDCC/HNTC/2011/40,Nguyễn Hải Hà,Đăng ký xe máy,2020-11-08 09:36:15.273,NaT,HĐ mở/đóng cuối ngày
1,399,TNN0254.01 Phố Chợ,2020-11-08,2020-11-08 07:34:14.677,2020-11-08 18:45:54.443,TNN0254,HDCC/TNN0254/2011/29,Đào Thiên Vũ,Đăng ký xe máy,2020-11-08 10:39:22.457,NaT,HĐ mở/đóng cuối ngày
2,164,F88 2A Nguyễn Khánh Toàn,2020-11-08,2020-11-08 08:23:39.960,2020-11-08 18:26:25.267,HN2ANKT,HDCC/HN2ANKT/2011/160,Nguyễn Thế Anh,Đăng ký xe máy,2020-11-08 15:37:41.470,NaT,HĐ mở/đóng cuối ngày
3,341,DNI0196.332 Hồng Thập Tự,2020-11-08,2020-11-08 08:17:12.660,2020-11-08 19:42:43.290,DNI0196,HDCC/DNI0196/2011/140,CHÌU THỊ LAN,Đăng ký xe máy,2020-11-08 17:18:28.707,NaT,HĐ mở/đóng cuối ngày
4,170,F88 218 Định Công,2020-11-08,2020-11-08 08:15:13.137,2020-11-08 21:19:43.837,HN218DC,HDCC/HN218DC/2011/58,Phan Lỗ Tấn,Xe máy,2020-11-08 20:53:13.217,NaT,HĐ mở/đóng cuối ngày


In [101]:
# Get ASM
shop_id_mkt='1ZTQE_pfBCAUr-0GSMPJBqtCyZVxX4hdiPuT_RDrdjeQ'
shop_wb_mkt=client.open_by_key(shop_id_mkt)
marketing_shop_data=shop_wb_mkt.worksheet_by_title('shop_detail').get_as_df()

In [104]:
marketing_shop_data = marketing_shop_data[['PGD', 'Mã PGD', 'GĐM', 'TPK', 'QLKV']]

In [121]:
# Get province risk level
# province_list_id
scope = ["https://spreadsheets.google.com/feeds",'https://www.googleapis.com/auth/spreadsheets',"https://www.googleapis.com/auth/drive.file","https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name(r'D:\F88\Python code\Immediate_warning\service_account.json', scope)
gc = gspread.authorize(credentials)
fault_group_id='1X7m2UnrVh909OKk9YOkxcIB455GPwTpAAcWGHIBn6_c'
fault_group_wb=gc.open_by_key(fault_group_id)
province_id = to_dataframe(fault_group_wb, 'province_list',1,0)

In [122]:
# Get ASM risk level
asm_risk_level = to_dataframe(fault_group_wb, 'asm_risk_level',1,0)

In [9]:
today=dt.date.today()
y_m=today.strftime('%y%m')

last_day=calendar.monthrange(today.year,today.month)[1]
#Lấy ngày cuối cùng của tháng
last_day_of_month=dt.datetime(today.year,today.month,last_day)

In [10]:
def get_n_days(nday):
    return (today+dt.timedelta(days=nday)).strftime("%Y-%m-%d")
number_day=-1

In [168]:
checkdate = (today+dt.timedelta(days = -1)).strftime("%Y-%m-%d")
yearmonth = (today+dt.timedelta(days = -1)).strftime("%Y%m")

In [160]:
province_risk_level = province_id[province_id['yearmonth'] == yearmonth]

In [161]:
asm_risk_level = asm_risk_level[asm_risk_level['yearmonth'] == yearmonth]

In [113]:
province_risk = shop_data.merge(province_risk_level, left_on = 'Tỉnh/TP', right_on = 'province_name', how = 'left')

In [115]:
province_risk = province_risk[['Mã PGD', 'province_name', 'province_risk_level']]

In [175]:
province_risk = province_risk[province_risk['Mã PGD'].isnull()==False]

In [123]:
asm_risk = shop_data_mkt.merge(asm_risk_level, left_on = 'QLKV', right_on = 'asm_name', how = 'left')

In [124]:
asm_risk = asm_risk[['Mã PGD', 'asm_name', 'risk_level']]

In [179]:
asm_risk = asm_risk.drop_duplicates()

In [181]:
asm_risk = asm_risk[asm_risk['asm_name'].isnull()==False]

In [12]:
diff_mins = '30'

In [79]:
#  outliers selection
def upper_outlier_selection(datacolumn):
    sorted(datacolumn)
    Q1, Q3 = np.percentile(datacolumn, [25,75])
    IQR = Q3 - Q1
#     lower_range = Q1 - (1.5 * IQR)
    upper_range = Q3 + (1.5 * IQR)
    return upper_range

HĐ Mở/Đóng trước khi kiểm kê/sau khi kiểm kê => 100% cần kiểm tra

In [169]:
checkdate

'2020-11-08'

In [170]:
open_close_before_after_cash_count_str = """Declare @check_date date = '"""+checkdate+"""'
  Declare @diff_mins int = '"""+diff_mins+"""'
  select main.*, case when (main.OpenHour <= main.min_cash_count
   or main.CloseHour <= main.min_cash_count) then N'HĐ đóng/mở trước khi kiểm kê lần đầu'
   when (main.OpenHour >= main.max_cash_count
   or main.Closehour >= main.max_cash_count) then N'HĐ đóng/mở sau khi kiểm kê lần cuối'
   when datediff(minute, main.min_cash_count, main.OpenHour) <= cast(@diff_mins as int)
   or datediff(minute, main.min_cash_count, main.CloseHour) <= cast(@diff_mins as int) then N'HĐ mở/đóng đầu ngày'
   when datediff(minute, main.max_cash_count, main.OpenHour) <= cast(@diff_mins as int) 
   or datediff(minute, main.max_cash_count, main.CloseHour) <= cast(@diff_mins as int) then N'HĐ mở/đóng cuối ngày'
   else 'Khác' end pawn_fraud_group
from (select cas1.*,pa.ShopCode, pa.CodeNo, Customer, cd.NAME category_name, occ.OpenHour, occ.CloseHour
from (select SHOP_ID,shop_name, cast(CREATED as date) cash_count_date, 
min(CREATED) min_cash_count,
max(CREATED) max_cash_count  
   from [dwh].[dbo].w_fund_daily_f 
   where cast(CREATED as date) = @check_date
  group by SHOP_ID,shop_name ,cast(CREATED as date) ) cas1 
  left join [dwh].[dbo].Pawn pa on pa.ShopID = cas1.SHOP_ID
  left join [dwh].[dbo].OPEN_CLOSE_CONTRACT_BY_TIME  occ on occ.CodeNo = pa.CodeNo
  left join [dwh].[dbo].W_CATEGORY_D cd on cd.CODE = pa.CategoryCode 
  where (cast(occ.OpenHour as date)= @check_date or cast(occ.CloseHour as date) = @check_date)) main
  """

In [171]:
# and (occ.OpenHour <= cas1.min_cash_count
#   or occ.OpenHour >= cas1.max_cash_count
#   or occ.Closehour <= cas1.min_cash_count
#   or occ.Closehour >= cas1.max_cash_count)

In [172]:
open_close_before_after_cash_count = pd.read_sql_query(open_close_before_after_cash_count_str,db)

In [182]:
open_close_before_after_cash_count = open_close_before_after_cash_count.merge(province_risk, left_on = 'shopcode', right_on='Mã PGD', how = 'left')

In [183]:
open_close_before_after_cash_count = open_close_before_after_cash_count.merge(asm_risk, left_on = 'shopcode', right_on='Mã PGD', how = 'left')

In [184]:
open_close_before_after_cash_count['province_risk_level'] = open_close_before_after_cash_count['province_risk_level'].fillna(0)

In [185]:
open_close_before_after_cash_count['province_risk_level'] = [int(i) for i in open_close_before_after_cash_count['province_risk_level']]

In [186]:
open_close_before_after_cash_count['risk_level'] = open_close_before_after_cash_count['risk_level'].fillna(0)

In [187]:
open_close_before_after_cash_count['risk_level'] = [int(i) for i in open_close_before_after_cash_count['risk_level']]

In [188]:
open_close_before_after_cash_count['total_risk_base'] = open_close_before_after_cash_count['province_risk_level'] + open_close_before_after_cash_count['risk_level']

In [189]:
# open_close_before_after_cash_count['fraud_group'] = np.where((open_close_before_after_cash_count['openhour']<=open_close_before_after_cash_count['min_cash_count'])|(open_close_before_after_cash_count['closehour']<=open_close_before_after_cash_count['min_cash_count']), 'HĐ đóng/mở trước khi kiểm kê',np.where((open_close_before_after_cash_count['openhour']>=open_close_before_after_cash_count['max_cash_count'])|(open_close_before_after_cash_count['closehour']>=open_close_before_after_cash_count['max_cash_count']), 'HĐ đóng/mở sau khi kiểm kê', 'Khác'))

In [212]:
open_close_before_after_cash_count['extract_date'] = checkdate
open_close_before_after_cash_count['year_month'] = yearmonth

In [190]:
open_close_before_after_cash_count['pawn_fraud_group'].unique()

array(['HĐ mở/đóng cuối ngày', 'HĐ đóng/mở trước khi kiểm kê lần đầu',
       'HĐ mở/đóng đầu ngày', 'HĐ đóng/mở sau khi kiểm kê lần cuối'],
      dtype=object)

In [213]:
full_sample_1 = open_close_before_after_cash_count[(open_close_before_after_cash_count['pawn_fraud_group'] == 'HĐ đóng/mở trước khi kiểm kê lần đầu')|(open_close_before_after_cash_count['pawn_fraud_group'] == 'HĐ đóng/mở sau khi kiểm kê lần cuối')]

In [214]:
# Danh sách HĐ đóng/mở đầu ngày cần chọn ra mẫu
part_sample_open_early = open_close_before_after_cash_count[(open_close_before_after_cash_count['pawn_fraud_group'] == 'HĐ mở/đóng đầu ngày')]

In [215]:
# Danh sách HĐ đóng/mở cuối ngày cần chọn ra mẫu
part_sample_open_late = open_close_before_after_cash_count[(open_close_before_after_cash_count['pawn_fraud_group'] == 'HĐ mở/đóng cuối ngày')]

In [216]:
part_sample_open_early.columns

Index(['shop_id', 'shop_name', 'cash_count_date', 'min_cash_count',
       'max_cash_count', 'shopcode', 'codeno', 'customer', 'category_name',
       'openhour', 'closehour', 'pawn_fraud_group', 'Mã PGD_x',
       'province_name', 'province_risk_level', 'Mã PGD_y', 'asm_name',
       'risk_level', 'total_risk_base', 'extract_date', 'year_month'],
      dtype='object')

In [228]:
check_early_part_1 = part_sample_open_early[(part_sample_open_early['total_risk_base'] ==10)|(part_sample_open_early['total_risk_base'] ==9)].sample(frac=0.5, replace=True, random_state=1)

In [229]:
check_early_part_2 = part_sample_open_early[(part_sample_open_early['total_risk_base'] ==8)|(part_sample_open_early['total_risk_base'] ==7)].sample(frac=0.2, replace=True, random_state=1)

In [240]:
check_late_part_1 = part_sample_open_late[(part_sample_open_late['total_risk_base'] ==10)|(part_sample_open_late['total_risk_base'] ==9)].sample(frac=0.2, replace=True, random_state=1)

In [236]:
check_late_part_2 = part_sample_open_late[(part_sample_open_late['total_risk_base'] ==8)|(part_sample_open_late['total_risk_base'] ==7)].sample(frac=0.05, replace=True, random_state=1)

In [241]:
check_late_part_1

,shop_id,shop_name,cash_count_date,min_cash_count,max_cash_count,shopcode,codeno,customer,category_name,openhour,...,pawn_fraud_group,Mã PGD_x,province_name,province_risk_level,Mã PGD_y,asm_name,risk_level,total_risk_base,extract_date,year_month
1233,34,F88 TH - 01 Ngô Quyền,2020-11-08,2020-11-08 08:05:00.297,2020-11-08 18:58:47.043,THH,HÐCC/THH/2009/149,Hoàng Giang,Đăng ký xe máy,NaT,...,HĐ mở/đóng cuối ngày,THH,Thanh Hóa,4,THH,Trần Đức Tiến,5,9,2020-11-08,202011
496,293,DNI0152.84 tổ 3 Đường Lý Thái Tổ,2020-11-08,2020-11-08 08:01:00.627,2020-11-08 19:26:02.643,DNI0152,HDCC/DNI0152/2011/121,PHẠM VĂN NGUYÊN,Đăng ký xe máy,2020-11-08 13:58:06.640,...,HĐ mở/đóng cuối ngày,DNI0152,Đồng Nai,5,DNI0152,Nguyễn Hoàng Huy,5,10,2020-11-08,202011
303,269,DNI0129.943 Phạm Văn Thuận,2020-11-08,2020-11-08 08:56:59.317,2020-11-08 20:53:45.590,DNI0129,HDCC/DNI0129/2011/127,ĐINH THỊ HUẾ THANH,Đăng ký xe máy,2020-11-08 11:24:49.010,...,HĐ mở/đóng cuối ngày,DNI0129,Đồng Nai,5,DNI0129,Nguyễn Hoàng Huy,5,10,2020-11-08,202011
316,341,DNI0196.332 Hồng Thập Tự,2020-11-08,2020-11-08 08:17:12.660,2020-11-08 19:42:43.290,DNI0196,HDCC/DNI0196/2011/133,BÌNH THỊ BÍCH THÙY,Đăng ký xe máy,2020-11-08 10:03:43.447,...,HĐ mở/đóng cuối ngày,DNI0196,Đồng Nai,5,DNI0196,Nguyễn Hoàng Huy,5,10,2020-11-08,202011
495,279,THA0139.33 Quang Trung,2020-11-08,2020-11-08 08:07:28.533,2020-11-08 19:01:43.363,THA0139,HDCC/THA0139/2011/24,Nguyễn Văn Vương,Đăng ký xe máy,2020-11-08 10:07:05.017,...,HĐ mở/đóng cuối ngày,THA0139,Thanh Hóa,4,THA0139,Trần Đức Tiến,5,9,2020-11-08,202011
231,341,DNI0196.332 Hồng Thập Tự,2020-11-08,2020-11-08 08:17:12.660,2020-11-08 19:42:43.290,DNI0196,HDCC/DNI0196/2011/140,CHÌU THỊ LAN,Đăng ký xe máy,2020-11-08 17:18:28.707,...,HĐ mở/đóng cuối ngày,DNI0196,Đồng Nai,5,DNI0196,Nguyễn Hoàng Huy,5,10,2020-11-08,202011
610,285,DNI0145.533 Phùng Hưng,2020-11-08,2020-11-08 08:15:56.300,2020-11-08 18:59:58.150,DNI0145,HDCC/DNI0145/2011/15,LÊ THỊ ÁNH,Đăng ký xe máy,2020-11-08 09:24:34.743,...,HĐ mở/đóng cuối ngày,DNI0145,Đồng Nai,5,DNI0145,Nguyễn Hoàng Huy,5,10,2020-11-08,202011
51,252,DNI0112.415 Đồng Khởi,2020-11-08,2020-11-08 08:36:51.240,2020-11-08 20:58:22.660,DNI0112,HDCC/DNI0112/2011/148,VŨ NGỌC LỘC,Đăng ký xe máy,2020-11-08 17:10:55.203,...,HĐ mở/đóng cuối ngày,DNI0112,Đồng Nai,5,DNI0112,Nguyễn Hoàng Huy,5,10,2020-11-08,202011


In [242]:
check_sample_1 = full_sample_1.append(check_early_part_1).append(check_early_part_2).append(check_late_part_1).append(check_late_part_2)

In [298]:
check_sample_1.columns

Index(['shop_id', 'shop_name', 'cash_count_date', 'min_cash_count',
       'max_cash_count', 'shopcode', 'codeno', 'customer', 'category_name',
       'openhour', 'closehour', 'pawn_fraud_group', 'Mã PGD_x',
       'province_name', 'province_risk_level', 'Mã PGD_y', 'asm_name',
       'risk_level', 'total_risk_base', 'extract_date', 'year_month'],
      dtype='object')

In [299]:
check_sample_1 = check_sample_1[['year_month','extract_date','shop_name','shopcode', 'customer', 'codeno','category_name','pawn_fraud_group']]

In [300]:
check_sample_1

,year_month,extract_date,shop_name,shopcode,customer,codeno,category_name,pawn_fraud_group
106,202011,2020-11-08,TNN0136.185 Lương Ngọc Quyến,TNN0136,Nguyễn Vũ Quang,HDCC/TNN0136/2011/21,Đăng ký Ô tô,HĐ đóng/mở trước khi kiểm kê lần đầu
399,202011,2020-11-08,TNN0136.185 Lương Ngọc Quyến,TNN0136,Dương Văn Hào,HDCC/TNN0136/2011/23,Đăng ký xe máy,HĐ đóng/mở trước khi kiểm kê lần đầu
499,202011,2020-11-08,F88 39 Hồ Tùng Mậu,HN39HTM,Hoàng Văn Thiện,HDCC/HN39HTM/2011/180,Điện thoại,HĐ đóng/mở sau khi kiểm kê lần cuối
560,202011,2020-11-08,TNN0136.185 Lương Ngọc Quyến,TNN0136,HOÀNG HỮU PHONG,HDCC/TNN0136/2011/22,Laptop,HĐ đóng/mở trước khi kiểm kê lần đầu
562,202011,2020-11-08,NBH0273.275 Nguyễn Công Trứ,NBH0273,Nguyễn Đức Lợi,HDCC/NBH0273/2011/1,Đăng ký xe máy,HĐ đóng/mở trước khi kiểm kê lần đầu
860,202011,2020-11-08,BDG0181.22 Tháng 12,BDG0181,TRẦN THỊ THỦY,HÐCC/BDG0181/2008/59,Đăng ký xe máy,HĐ đóng/mở sau khi kiểm kê lần cuối
930,202011,2020-11-08,F88 2A Nguyễn Khánh Toàn,HN2ANKT,Nguyễn Khả Bảo,HDCC/HN2ANKT/2011/166,Laptop,HĐ đóng/mở sau khi kiểm kê lần cuối
1248,202011,2020-11-08,BDG0127.89 đường N5,BDG0127,BÙI THỊ DIỆN,HDCC/BDG0127/2011/142,Đăng ký xe máy,HĐ đóng/mở sau khi kiểm kê lần cuối
1293,202011,2020-11-08,F88 2A Nguyễn Khánh Toàn,HN2ANKT,Nguyễn Khả Bảo,HÐCC/HN2ANKT/2010/130,Laptop,HĐ đóng/mở sau khi kiểm kê lần cuối
1218,202011,2020-11-08,DNI0152.84 tổ 3 Đường Lý Thái Tổ,DNI0152,NGUYỄN THANH PHƯƠNG,HDCC/DNI0152/2011/111,Đăng ký xe máy,HĐ mở/đóng đầu ngày


- HĐ đóng/mở cuối ngày: đóng/mở trong vòng 30 phút trước khi kiểm kê lần cuối trong ngày
- HĐ đóng/mở đầu ngày: đóng/mở trong vòng 30 phút sau khi kiểm kê lần đầu trong ngày

In [302]:
#  Khoảng thời gian mở HĐ
open_str = """Declare @ext_date date= '"""+checkdate+"""'
select le.EXTRAC_DATE,support_staff,created_staff,ShopCode,le.CodeNo,Customer,
NAME categoryname,OpenHour,InitTime,complete, shop_name 
from [dwh].[dbo].OPEN_CLOSE_CONTRACT_BY_TIME le 
left join [dwh].[dbo].W_CATEGORY_D cate on cate.CODE=le.CategoryCode 
left join (select codeno,FirstSupportID,u.USER_CODE support_staff,u2.USER_CODE created_staff,
ShopCode,InitTime, sd.name shop_name
from [dwh].[dbo].pawn pa1 
left join [dwh].[dbo].ShopDetail sd on pa1.shopcode = sd.code
left join [dwh].[dbo].W_USER_F u on pa1.firstsupportid=u.USER_WID 
left join [dwh].[dbo].W_USER_F u2 on pa1.CreatedBy=u2.USER_WID 
where pa1.status not in (15,22,11))pa on le.CodeNo=pa.CodeNo 
left join (select PAWN_WID,CONTRACT_NO,min(CREATED)complete from [dwh].[dbo].W_PAWN_TRANSACTION_F 
where action_name like N'Cho vay' 
group by PAWN_WID,CONTRACT_NO)trans 
on trans.CONTRACT_NO=le.CodeNo where OpenHour is not null and cast(OpenHour as date) = @ext_date"""

In [303]:
open_data=pd.read_sql_query(open_str,db)

In [304]:
open_data['init_to_comple']=(pd.to_datetime(open_data['complete'])-open_data.inittime).dt.seconds/60

In [305]:
full_sample_threshold = pd.read_excel(r'D:\F88\Python code\Immediate_warning\official job\Optimize\pawn_process_theshold.xlsx', sheet_name='full_sample')
part_sample_threshold_short = pd.read_excel(r'D:\F88\Python code\Immediate_warning\official job\Optimize\pawn_process_theshold.xlsx', sheet_name='part_process_short')
part_sample_threshold_long = pd.read_excel(r'D:\F88\Python code\Immediate_warning\official job\Optimize\pawn_process_theshold.xlsx', sheet_name='part_process_long')

In [306]:
part_sample_threshold_short.columns = ['category_name', 'short_min', 'short_max']

In [307]:
part_sample_threshold_long.columns = ['category_name', 'long_min', 'long_max']

In [308]:
open_data = open_data.merge(full_sample_threshold, left_on = 'categoryname',right_on =  'category_name',how = 'left')

In [309]:
open_data = open_data.merge(part_sample_threshold_short, left_on = 'categoryname',right_on =  'category_name',how = 'left')

In [310]:
open_data = open_data.merge(part_sample_threshold_long, left_on = 'categoryname',right_on =  'category_name',how = 'left')

In [311]:
open_data.columns

Index(['extrac_date', 'support_staff', 'created_staff', 'shopcode', 'codeno',
       'customer', 'categoryname', 'openhour', 'inittime', 'complete',
       'shop_name', 'init_to_comple', 'index', 'category_name_x',
       'lower_threshold', 'upper_threshold', 'category_name_y', 'short_min',
       'short_max', 'category_name', 'long_min', 'long_max'],
      dtype='object')

In [312]:
#  Tất cả các HĐ được cảnh báo khi init_to_comple <lower_threshold hoặc init_to_comple >upper_threshold
full_sample_2a = open_data[open_data['init_to_comple']<=open_data['lower_threshold']]
full_sample_2b = open_data[open_data['init_to_comple']>=open_data['upper_threshold']]

In [313]:
full_sample_2a['pawn_fraud_group'] = 'HĐ mở trong thời gian ngắn bất thường'
full_sample_2b['pawn_fraud_group'] = 'HĐ mở trong thời gian dài bất thường'

In [314]:
full_sample_2 = full_sample_2a.append(full_sample_2b)

In [315]:
full_sample_2a1 = full_sample_2[(full_sample_2['categoryname'] == 'Đăng ký Ô tô')|(full_sample_2['categoryname'] == 'Đăng ký xe máy')]

In [316]:
full_sample_2b1 = full_sample_2[(full_sample_2['categoryname'] == 'Điện thoại')|(full_sample_2['categoryname'] == 'Xe máy')|(full_sample_2['categoryname'] == 'Laptop')|(full_sample_2['categoryname'] == 'Ô tô')]

In [317]:
# Chọn mẫu check HĐ mở trong thời gian ngắn bất thường theo tỷ lệ check
part_sample_short = open_data[(open_data['init_to_comple']>open_data['short_min']) & (open_data['init_to_comple']<open_data['short_max'])]

In [318]:
part_sample_short['pawn_fraud_group'] = 'HĐ mở trong thời gian ngắn bất thường'

In [319]:
# Chọn mẫu check HĐ mở trong thời gian dài bất thường theo tỷ lệ check
part_sample_long = open_data[(open_data['init_to_comple']>open_data['long_min']) & (open_data['init_to_comple']<open_data['long_max'])]

In [320]:
part_sample_long['pawn_fraud_group'] = 'HĐ mở trong thời gian dài bất thường'

In [321]:
check_full_sample_2b = full_sample_2b1.sample(frac=0.3, replace=True, random_state=1)

In [322]:
check_part_sample_long = part_sample_long.sample(frac=0.3, replace=True, random_state=1)

In [323]:
check_part_sample_short = part_sample_short.sample(frac=0.5, replace=True, random_state=1)

In [324]:
check_sample_2 = full_sample_2a1.append(check_full_sample_2b).append(check_part_sample_long).append(check_part_sample_short)

In [326]:
check_sample_2['extract_date'] = checkdate
check_sample_2['year_month'] = yearmonth

In [327]:
check_sample_2.columns

Index(['extrac_date', 'support_staff', 'created_staff', 'shopcode', 'codeno',
       'customer', 'categoryname', 'openhour', 'inittime', 'complete',
       'shop_name', 'init_to_comple', 'index', 'category_name_x',
       'lower_threshold', 'upper_threshold', 'category_name_y', 'short_min',
       'short_max', 'category_name', 'long_min', 'long_max',
       'pawn_fraud_group', 'extract_date', 'year_month'],
      dtype='object')

In [328]:
check_sample_2 = check_sample_2[['year_month','extract_date','shop_name','shopcode', 'customer', 'codeno','category_name','pawn_fraud_group']]

In [343]:
#  Đóng HĐ trong vòng 5 ngày từ ngày mở (càng đóng sớm càng có vấn đề)
closed_within_5_days_str = """Declare @check_date date = '"""+checkdate+"""'
  select pa.Pawnid, pa.CodeNo,pa.shopcode, sd.Name shopname, cus.Name customer_name, pa.FromDate, pa.Todate ,pa.CloseDate, 
  DATEDIFF(day, pa.FromDate, pa.CloseDate) real_loan_days, cate.NAME category_name
  from pawn pa
  left join ShopDetail sd on pa.ShopCode=sd.Code
  left join Customer cus on cus.CustomerID = pa.CustomerID
  left join W_CATEGORY_D cate on cate.CODE = pa.CategoryCode 
  where pa.CloseDate is not null
  and DATEDIFF(day, pa.fromdate, pa.CloseDate) <=5
  and pa.closedate = @check_date
  and pa.STATUS not in (11,15,22)"""

In [344]:
closed_within_5_days = pd.read_sql_query(closed_within_5_days_str,db)

In [345]:
closed_within_5_days['category_name'].unique()

array(['Đăng ký xe máy', 'Điện thoại', 'Laptop', 'Xe máy', 'Trang sức',
       'Đăng ký Ô tô', 'Đồng hồ', 'Máy tính bảng'], dtype=object)

In [346]:
check_closed_within_5_days = closed_within_5_days[(closed_within_5_days['category_name'] == 'Đăng ký xe máy')|(closed_within_5_days['category_name'] == 'Xe máy')|(closed_within_5_days['category_name'] == 'Đăng ký Ô tô')|(closed_within_5_days['category_name'] == 'Ô tô')]

In [347]:
check_sample_3 = check_closed_within_5_days.sample(frac=0.5, replace=True, random_state=1)

In [348]:
check_sample_3['extract_date'] = checkdate
check_sample_3['yearmonth'] = yearmonth

In [356]:
check_sample_3.columns = ['pawnid', 'codeno', 'shopcode', 'shop_name', 'customer', 'fromdate',
       'todate', 'closedate', 'real_loan_days', 'category_name',
       'extract_date', 'year_month']

In [ ]:
check_sample_3['pawn_fraud_group'] = 'HĐ đóng trong vòng 5 ngày kể từ ngày mở'

In [357]:
check_sample_3 = check_sample_3[['year_month','extract_date','shop_name','shopcode', 'customer', 'codeno','category_name','pawn_fraud_group']]

In [38]:
#  HĐ TS thường thanh toán khi quá hạn trên 5 ngày
overdue_close_str= """Declare @ext_date date ='"""+get_n_days(number_day)+"""' 
select pa.Pawnid,shop.Name shopname,pa.ShopCode,po.CustomerName,
pa.CodeNo,po.CategoryName,pa.FromDate,pa.CloseDate,pa.MoneyCurrent,po.TuoiNo+1 dpd 
from pawn pa 
left join PawnOverdue po on (po.PawnID=pa.Pawnid and po.CREATED=dateadd(day,-1,pa.CloseDate)) 
left join ShopDetail shop on pa.ShopCode=shop.Code 
where pa.STATUS =1 and pa.CloseDate=@ext_date 
and po.CategoryName not like N'%Đăng ký%' 
and TuoiNo>5 and pa.ShopCode not like '%DR%'
and pa.ShopCode not like '%TEST%'
and pa.ShopCode not like '%HS%'"""

In [39]:
overdue_close_data=pd.read_sql_query(overdue_close_str,db)

In [288]:
overdue_close_data['pawn_fraud_group'] = 'HĐ thanh toán khi quá hạn trên 5 ngày'

In [359]:
overdue_close_data['extract_date'] = checkdate
overdue_close_data['year_month'] = yearmonth

In [362]:
overdue_close_data.columns = ['pawnid', 'shop_name', 'shopcode', 'customer', 'codeno',
       'category_name', 'fromdate', 'closedate', 'moneycurrent', 'dpd',
       'pawn_fraud_group', 'extract_date', 'year_month']

In [363]:
check_sample_4 = overdue_close_data[['year_month','extract_date','shop_name','shopcode', 'customer', 'codeno','category_name','pawn_fraud_group']]

In [378]:
# LTV tại thời điểm giải ngân
ltv_result_str ="""Declare @check_date date = '"""+checkdate+"""'
  select CategoryName,CustomerName, ShopName,pawnid, ContractCode,shopcode,
  case when PackageCode like '%NHANH%' then N'Vay nhanh' else N'Vay thường' end package_name,
  (moneyOrg - InsurranceMoney)/MoneyAppraisal ltv_index
  from [dwh].[dbo].[PawnOverdue]
  where CREATED = @check_date
  and FromDate = @check_date"""

In [379]:
ltv_of_pawn = pd.read_sql_query(ltv_result_str, db)

In [380]:
ltv_title = ltv_of_pawn[(ltv_of_pawn['categoryname'] =='Đăng ký xe máy')|(ltv_of_pawn['categoryname'] =='Đăng ký Ô tô')]

In [381]:
check_ltv = ltv_title[ltv_title['ltv_index']>=0.75]

In [382]:
check_ltv['pawn_fraud_group'] = 'HĐ đăng ký có LTV cao'
check_ltv['extract_date'] = checkdate
check_ltv['year_month'] = yearmonth

In [383]:
check_ltv.columns = ['category_name', 'customer', 'shop_name', 'pawnid', 'codeno',
       'shopcode', 'package_name', 'ltv_index', 'pawn_fraud_group',
       'extract_date', 'year_month']

In [384]:
check_sample_5 = check_ltv[['year_month','extract_date','shop_name','shopcode', 'customer', 'codeno','category_name','pawn_fraud_group']]

In [385]:
full_sample_check = check_sample_1.append(check_sample_2).append(check_sample_3).append(check_sample_4).append(check_sample_5)

In [390]:
full_sample_check['risk_type'] ='Gian lận'

In [391]:
write_data(control_wb,'Checking','Checking',full_sample_check)

Đã ghi dữ liệu lên sheet Checking


In [54]:
# #  Tỷ lệ giải ngân CK, tiền mặt - Đối với sp ĐK
# cash_credit_disbur_str = """Declare @check_date date = '"""+checkdate+"""'
# select case when CategoryCode like '%15%' then N'Đăng ký Ô tô' 
# else N'Đăng ký xe máy' end category_name, pawnid, CodeNo, (LoanMoneyOrg - InsurranceMoney) net_disbur, CashMoney, CreditMoney 
# from [dwh].[dbo].pawn
# where (CategoryCode like '%15%' or CategoryCode like '%17%')
# and ShopCode not like '%HS%'
# and ShopCode not like '%TEST%'
# and FromDate = @check_date
# and status in (1, 14)"""

In [55]:
# cash_credit_disbur = pd.read_sql_query(cash_credit_disbur_str, db)

In [57]:
# cash_credit_disbur = cash_credit_disbur.fillna(0)

In [58]:
# cash_credit_disbur.columns

Index(['category_name', 'pawnid', 'codeno', 'net_disbur', 'cashmoney',
       'creditmoney'],
      dtype='object')

In [59]:
# cash_credit_disbur['cash_rate'] = cash_credit_disbur['cashmoney']/cash_credit_disbur['net_disbur']

In [61]:
# import matplotlib.pyplot as plt
# import seaborn as sns

In [81]:
# # Cảnh báo thử nghiệm các HĐ giải ngân thuộc nhóm outlier
# motor_title_outlier = upper_outlier_selection(cash_credit_disbur[cash_credit_disbur['category_name'] =='Đăng ký xe máy']['net_disbur'])

In [82]:
# car_title_outlier = upper_outlier_selection(cash_credit_disbur[cash_credit_disbur['category_name'] =='Đăng ký Ô tô']['net_disbur'])

In [84]:
# #  Cảnh báo với HĐ DKXM giải ngân tiền mặt 100%
# cash_credit_disbur[(cash_credit_disbur['category_name'] =='Đăng ký xe máy') & (cash_credit_disbur['net_disbur'] >motor_title_outlier)]

,category_name,pawnid,codeno,net_disbur,cashmoney,creditmoney,cash_rate
38,Đăng ký xe máy,1560515,HDCC/HBH0247/2011/40,27700000.0,900000.0,26800000.0,0.032491
68,Đăng ký xe máy,1560157,HDCC/VY/2011/116,26900000.0,26900000.0,0.0,1.000000
79,Đăng ký xe máy,1560178,HDCC/VY/2011/119,30000000.0,30000000.0,0.0,1.000000
212,Đăng ký xe máy,1560312,HDCC/BTE0184/2011/130,27200000.0,27200000.0,0.0,1.000000
224,Đăng ký xe máy,1560659,HDCC/HN2ANKT/2011/160,30000000.0,19214000.0,10786000.0,0.640467
257,Đăng ký xe máy,1560714,HDCC/DNI0145/2011/17,29500000.0,29500000.0,0.0,1.000000
335,Đăng ký xe máy,1560763,HDCC/SGN0141/2011/145,30000000.0,30000000.0,0.0,1.000000
345,Đăng ký xe máy,1560786,HDCC/DTP0250/2011/29,30000000.0,30000000.0,0.0,1.000000
422,Đăng ký xe máy,1560904,HDCC/HN100BT/2011/128,30000000.0,30000000.0,0.0,1.000000
475,Đăng ký xe máy,1561019,HDCC/SGN0121/2011/155,27200000.0,27200000.0,0.0,1.000000


In [85]:
# #  Cảnh báo với HĐ DKOT giải ngân tiền mặt 100%
# cash_credit_disbur[(cash_credit_disbur['category_name'] =='Đăng ký Ô tô') & (cash_credit_disbur['net_disbur'] >car_title_outlier)]

,category_name,pawnid,codeno,net_disbur,cashmoney,creditmoney,cash_rate
28,Đăng ký Ô tô,1560495,HDCC/SG2001/2011/113,299900000.0,2065000.0,297835000.0,0.006886
47,Đăng ký Ô tô,1560531,HDCC/SG2010/2011/146,300000000.0,0.0,300000000.0,0.000000
464,Đăng ký Ô tô,1561000,HDCC/THA0272/2011/43,320000000.0,0.0,320000000.0,0.000000
